# A slightly less naive and foolish keras model

(Not working yet)


In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(123)

from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.layers.recurrent import LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

from preprocess import *
from util import *

In [8]:
PITCH_LEN = len(pitchToTuple(SILENCE))
ARBITRARY_DIMENSIONS=12
MIN_SONG_LENGTH=200 #TODO figure out/look up

model = Sequential([
        Embedding(PITCH_LEN, ARBITRARY_DIMENSIONS, input_length=MIN_SONG_LENGTH),
        LSTM(ARBITRARY_DIMENSIONS, return_sequences=True),
        # Dropout(0.5),
        TimeDistributed(Dense(89)),
        Activation('sigmoid')
        ])
# Binary crossentropy?
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])



In [10]:
# Load background data
for song_number in tqdm(range(500, 502)): # 500-1000
    preprocessSong(song_number)

100%|██████████| 2/2 [00:09<00:00,  4.67s/it]


In [11]:
data = loadedData[0]
labels = loadedData[1]
min_song_length=loadedData[2]

model.fit(data, labels, nb_epoch=10, batch_size=32)

NameError: name 'loadedData' is not defined

In [47]:
# def predict_harmony(melody_pitches_so_far):
#     return tupleToPitch([model.predict([melody_pitches_so_far])])


# list(map(predict_harmony, data))

[tupleToPitch(n) for n in model.predict(data)]

[20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
